In [ ]:
BEGIN TRAN
--<full code>
--select * from backupTableRef
--select * from golfnowone.dbo.<tableUpdatedOrDeleted>
ROLLBACK [COMMIT]

In [ ]:
USE GolfNowOne
GO

SELECT
    TOP 50
    COUNT(OI.ID), OI.OrderID, S.Name
FROM [OrderItem] AS OI
INNER JOIN [Order] AS O ON OI.OrderID = O.ID AND O.SourceID = 5
INNER JOIN [Spa] AS S ON O.SpaID = S.ID
GROUP BY OI.OrderID, S.Name
HAVING COUNT(OI.ID) > 1
ORDER BY COUNT(OI.ID) DESC

In [ ]:
USE GolfNowOne
GO
;

SELECT
    C.ID
    ,C.FirstName
    ,C.LastName
    ,C.Email
    ,C.GUID
    ,C.DateLastModified
    ,S.ID AS [LocationID]
    ,S.Name AS [Location]
    ,B.ID AS [BrandID]
    ,B.Name AS [Brand]
FROM Customer AS C
INNER JOIN Spa AS S ON C.SpaID = S.ID AND S.ID = 36044
LEFT JOIN Brand AS B ON S.BrandID = B.ID
WHERE C.FirstName NOT LIKE 'First%' AND C.LastName NOT LIKE 'Last%'

In [ ]:
USE GolfNowOne
GO
;

SELECT
*
FROM [dbo].[dbverinfo]
ORDER BY EventDate DESC